In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pyautogui

import csv
import json
import time

from tqdm.notebook import tqdm

class StockVid(object):
    def __init__(self, stock, date, views):
        self.stock = stock
        self.date = date
        self.views = views
    
    def jsonEnc(self):
      return {'stock': self.stock, 'date': self.date, 'views': self.views}

def jsonDefEncoder(obj):
    if hasattr(obj, 'jsonEnc'):
        return obj.jsonEnc()
    else: #some default behavior
        return obj.__dict__

    
### DATABASE FUNCTIONS ###

    # returns connection object #
def connect_to_db(db_name):
    cnx = mysql.connector.connect(
    user='root',
    password='chalkHorseMountain',
    host='localhost',
    database=db_name
    )
    return cnx
    
    # returns boolean #
def table_exists(cursor, tbl_name):
    cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_schema = DATABASE()
        AND table_name = \"""" + tbl_name + """\";
    """)
    
    if cursor.fetchone()[0] == 1:
        return True
    return False
     
    
class youtubeScraper:

    def get_vids(self, stock):
        
        stockTickers = {}
        stockTickers[stock] = {}
        relevantVids = []
        
        driver = webdriver.Chrome('C:/WebDriver/chromedriver')
        driver.get("https://www.youtube.com/results?search_query=" + stock)
       
    
        ## Begin Scrolling ##
        start = time.perf_counter()
        print("Phase[1/2]--Scrolling", end=' ')
        while(True):
            try:
                driver.find_element_by_xpath("//*[@id='message'][text()='No more results']")
                break
            except:
                pyautogui.press('pagedown')
                continue
             
            break
            
        end = time.perf_counter()
        p1_time = divmod(int(end-start), 60)
        print("[{:02}:{:02}]".format(p1_time[0], p1_time[1]))
        
            
        ## Begin Scraping ##
        cnx = connect_to_db("TheSpatula")
        mycursor = cnx.cursor()
        assert mycursor
        assert table_exists(mycursor, "reddit")
        
        all_vids = driver.find_elements_by_xpath("//div[@ID='contents']/ytd-video-renderer")
        
        for i in tqdm(range(len(all_vids)), desc = 'Phase [2/2] -- Scraping'):
            vid_views = all_vids[i].find_element_by_xpath(".//*[@id='metadata-line']/span[1]").text
            
            try:
                vid_date = all_vids[i].find_element_by_xpath(".//*[@id='metadata-line']/span[2]").text
            except:
                vid_date = "live"
                
            stockTickers[stock][i] = StockVid(stock, vid_date, vid_views) 
            
        driver.close()
        
        ## Begin Formatting Results ##
        if(len(stockTickers[stock]) > 0):
            for post in stockTickers[stock]:
                relevantVids.append(stockTickers[stock][post])
        json_object = json.dumps(relevantVids, default=jsonDefEncoder, indent = 4)
        
        print(json_object) 
        
        
if __name__ == "__main__":
    youtubeScraper().get_vids('safemoon')

Phase[1/2]--Scrolling [01:04]



[
    {
        "stock": "safemoon",
        "date": "6 hours ago",
        "views": "16K views"
    },
    {
        "stock": "safemoon",
        "date": "2 hours ago",
        "views": "5.8K views"
    },
    {
        "stock": "safemoon",
        "date": "5 hours ago",
        "views": "2K views"
    },
    {
        "stock": "safemoon",
        "date": "10 hours ago",
        "views": "29K views"
    },
    {
        "stock": "safemoon",
        "date": "9 hours ago",
        "views": "12K views"
    },
    {
        "stock": "safemoon",
        "date": "Streamed 2 hours ago",
        "views": "7K views"
    },
    {
        "stock": "safemoon",
        "date": "Streamed 2 hours ago",
        "views": "2.6K views"
    },
    {
        "stock": "safemoon",
        "date": "2 days ago",
        "views": "21K views"
    },
    {
        "stock": "safemoon",
        "date": "1 day ago",
        "views": "45K views"
    },
    {
        "stock": "safemoon",
        "date": "3 days ago"

In [ ]:
## Creating youtube_<stock> Table
mycursor.execute("""
    CREATE TABLE youtube_""" + stock + """ (
        post_id INT, num_views INT, date_posted DATE
    );
    """)